In [49]:
import pandas as pd
from datetime import datetime,timedelta
import yfinance as yf

In [2]:
transactions_df = pd.read_csv("Resources/Transactions.csv", parse_dates=True, infer_datetime_format=True)


In [3]:
transactions_df.head()

,Date,Amount,Currency,Transaction
0,2021-08-07,1,BTC,BUY
1,2021-08-08,5,ETH,BUY
2,2021-08-08,20,SOL,BUY
3,2021-08-11,15,SOL,SELL


In [61]:
def get_price(date,currency):
    pair=currency+"-AUD"
    data = yf.download(pair,date,date + timedelta(days=1))
    return data.iloc[0,4]
    
def calculate_daily_value(date,summary_portfolio_df):
    amount_aud = 0
    for index, row in summary_portfolio_df.iterrows(): 
        aud_price = get_price(date,index)
        amount_aud += row['Amount'] * aud_price
        daily_summary = pd.DataFrame({'Date':[date], 'AUD':[amount_aud]})
    
    return daily_summary

In [62]:
last_date=0
summary_portfolio_df = pd.DataFrame(columns=['Currency','Amount'])
summary_portfolio_df.set_index('Currency',inplace=True)
daily_portfolio = pd.DataFrame(columns=['Date','AUD'])

for index, row in transactions_df.iterrows():
    #parse the row
    date = datetime.strptime(row['Date'], '%Y-%m-%d')
    
    if last_date != date:
        #Date changed, summarize the day value (ecepet for the first run)
        if index > 0:
            daily_portfolio =  pd.concat([daily_portfolio,calculate_daily_value(last_date,summary_portfolio_df)])
        last_date = date
        
    currency = row['Currency']
    if row['Transaction'] == 'SELL':
        amount = -row['Amount']
    else:
        amount = row['Amount']
        
    if not currency in summary_portfolio_df.index:
        new_row = pd.DataFrame({'Currency':[currency], 'Amount':[amount]})
        new_row.set_index('Currency',inplace=True)
        summary_portfolio_df = pd.concat([summary_portfolio_df,new_row])
    else:
        summary_portfolio_df.loc[currency, 'Amount'] += amount   
    
    #If last day then summarize
    if index == len(transactions_df)-1:
        daily_portfolio =  pd.concat([daily_portfolio,calculate_daily_value(last_date,summary_portfolio_df)])

daily_portfolio.set_index('Date',inplace=True)
display(summary_portfolio_df.head())
display(daily_portfolio.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Amount
Currency,
BTC,1
ETH,5
SOL,5


,AUD
Date,
2021-08-07,60584.707031
2021-08-08,81278.748123
2021-08-11,83592.386787


In [ ]:
#1. Generate all dates
transactions_df.sort_values(
start_date=transactions_df.iloc[0,0]
end_date=transactions_df.iloc[0,0]

#2. For every date
    #1. Check if transactions exists:
        # if yes, update total balance of coins
    #2. Calcualte AUD balance
    